### Entities:

	1.	Book: Each book has a title and an author.
	2.	Member: Each member can borrow multiple books.
	3.	Borrow: Tracks when a member borrows and returns a book (with borrow/return dates).




### Define the Models

	1.	Define Book, Member, and Borrow.
	2.	Implement foreign key relationships.

In [1]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Date, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker


In [2]:
Base = declarative_base()

In [3]:
class Book(Base):
    
    __tablename__ = "books"
    
    id = Column(Integer, primary_key = True)
    title = Column(String, nullable=False)
    author = Column(String, nullable=False)

In [4]:
class Member(Base):
    
    __tablename__ = "members"
    
    id = Column(Integer, primary_key = True)
    name = Column(String, nullable=False)

In [5]:
class Borrow(Base):
    
    __tablename__ = "borrows"
    
    id = Column(Integer, primary_key = True)
    
    book_id = Column(Integer, ForeignKey("books.id"))
    member_id = Column(Integer, ForeignKey("members.id"))
    
    borrow_date = Column(Date)
    return_date = Column(Date)
    
    book = relationship('Book')
    member = relationship('Member')


In [6]:
engine = create_engine("sqlite:///library.db")

In [7]:
Base.metadata.create_all(engine)

In [8]:
Session = sessionmaker(bind = engine)

In [9]:
session = Session()

### Insert Sample Data

In [10]:
from datetime import date

In [11]:
book1 = Book(title = 'Harry Potter', author = 'J.K. Rowling')
book2 = Book(title = 'Harry Potter', author = 'J.K. Rowling')
book3 = Book(title = 'Atomic Habits', author = 'James Clear')


member1 = Member(name = 'Nathan')
member2 = Member(name = 'Caslow')
member3 = Member(name = 'Yuke')

borrow1 = Borrow(book = book1, member = member1, borrow_date = date(2024, 9, 1))
borrow2 = Borrow(book = book2, member = member2, borrow_date = date(2024, 9, 2))
borrow3 = Borrow(book = book3, member = member3, borrow_date = date(2024, 9, 3))


In [12]:
session.add_all([book1, book2, book3, member1, member2, member3,borrow1, borrow2, borrow3])
session.commit()

### Query Data

In [14]:
for book in session.query(Book).all():
    print(book.title, book.author)

Harry Potter J.K. Rowling
Harry Potter J.K. Rowling
Atomic Habits James Clear


In [15]:
for book in session.query(Book).filter(Book.title == 'Harry Potter').all():
    print(book.title, book.author)

Harry Potter J.K. Rowling
Harry Potter J.K. Rowling


### Query all books borrowed by a specific member


In [17]:
def query_books_by_member(member_name):
    
    member_id = session.query(Member).filter(Member.name == member_name).first().id
    
    if member_id:
        
        book_id = session.query(Borrow).filter(Borrow.member_id == member_id).first().book_id
        
        book_name = session.query(Book).filter(Book.id == book_id).first().title
        
        print(book_name)
        
query_books_by_member('Nathan')

Harry Potter


### Return Books

- Mark a borrowed book as returned by updating the `return_date`.
 

In [18]:
def return_book(borrow_id, return_date):
    
    borrow = session.query(Borrow).filter(Borrow.id == borrow_id).first()
    
    if borrow:
    
        borrow.return_date = return_date
        
        print(f'{borrow_id} updated on {return_date}')

In [19]:
return_book(1, date(2024, 9, 10))

1 updated on 2024-09-10


In [20]:
session.commit()

### Find all currently available books in the library

In [31]:
borrowed_book_ids = session.query(Borrow).filter(Borrow.return_date == None).all()

borrowed_book_ids = [borrow.book_id for borrow in borrowed_book_ids]


available_books = session.query(Book).filter(Book.id.notin_(borrowed_book_ids)).all()

for book in available_books:
    print(book.title)

Harry Potter


In [24]:
query = """SELECT title, author FROM books WHERE id not in (SELECT book_id FROM borrows WHERE return_date is NULL);"""

In [25]:
from sqlalchemy import text


result = session.execute(text(query)).fetchall()

In [27]:
print([res for res in result])

[('Harry Potter', 'J.K. Rowling')]
